In [ ]:
!nvidia-smi

Wed Jun 30 13:02:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Data

In [ ]:
import pandas as pd
df_typo = pd.read_csv("/content/drive/MyDrive/HateSpeech/TODA(CLEANED)/BERT/TODA_TYPO_01.csv").iloc[:,1:]
df_typo = df_typo.dropna()
X_cleaned = df_typo['text_cleaned']
X_typo = df_typo['typo_corrected']
df_typo = df_typo[~((X_typo.duplicated())|X_cleaned.duplicated())]
X_cleaned = df_typo['text_cleaned']
X_typo = df_typo['typo_corrected']
y = df_typo['class']
print(df_typo.shape)
df_typo = df_typo.reset_index(drop=True)
df_typo

(317802, 7)


,text_raw,text_cleaned,typo_corrected,tokens_raw,tokens_stop,tokens_stems,class
0,@BreitbartNews Idiots seem to come out of the ...,idiots seem to come out of the woodwork it h...,idiots seem to come out of the woodwork it h...,"[' ', 'idiots', 'seem', 'to', 'come', 'out', '...","['idiots', 'come', 'woodwork', 'threatens', 'p...","['idiot', 'come', 'woodwork', 'threaten', 'pre...",1
1,russia as a huge black hole where everything d...,russia as a huge black hole where everything d...,russia as a huge black hole where everything d...,"['russia', 'as', 'a', 'huge', 'black', 'hole',...","['russia', 'huge', 'black', 'hole', 'disappear...","['russia', 'huge', 'black', 'hole', 'disappear...",0
2,@user so much stuff happening in florida! firs...,so much stuff happening in florida! first orl...,so much stuff happening in florida! first orl...,"[' ', 'so', 'much', 'stuff', 'happening', 'in'...","['stuff', 'happening', 'florida', 'orlando', '...","['stuff', 'happen', 'florida', 'orlando', 'sho...",0
3,"@user #pennydreadful a long, dark love poem ð...",pennydreadful a long dark love poem i agree...,pennydreadful a long dark love poem i agree...,"[' ', 'pennydreadful', 'a', 'long', 'dark', '...","['pennydreadful', 'long', 'dark', 'love', 'poe...","['pennydread', 'long', 'dark', 'love', 'poem',...",0
4,@user @user @user this is how i wanted to spnd...,this is how i wanted to spnd father's day w...,this is how i wanted to send father's day w...,"[' ', 'this', 'is', 'how', 'i', 'wanted', 't...","['wanted', 'send', 'father', 'day', 'wind', 'b...","['want', 'send', 'father', 'day', 'wind', 'boy...",0
...,...,...,...,...,...,...,...
317797,.@J0EKAY never in my life did I think I'd hear...,never in my life did i think i'd hear rhythm ...,never in my life did i think i'd hear rhythm ...,"[' ', 'never', 'in', 'my', 'life', 'did', 'i',...","['life', 'think', 'hear', 'rhythm', 'n', 'gash...","['life', 'think', 'hear', 'rhythm', 'n', 'gash...",0
317798,"whereas East Asians came from East Africa,",whereas east asians came from east africa,whereas east asians came from east africa,"['whereas', 'east', 'asians', 'came', 'from', ...","['east', 'asians', 'came', 'east', 'africa']","['east', 'asian', 'came', 'east', 'africa']",0
317799,@AndrewBLeh @N7Kopper @nitramy Ever play Rambo...,ever play rambo on old nes by any chance?,ever play rambo on old nes by any chance?,"[' ', 'ever', 'play', 'rambo', 'on', 'old', ...","['play', 'rambo', 'old', 'nes', 'chance']","['play', 'rambo', 'old', 'ne', 'chanc']",1
317800,Merger \n\nIf this article is to avoid being m...,merger if this article is to avoid being merge...,merger if this article is to avoid being merge...,"['merger', 'if', 'this', 'article', 'is', 'to'...","['merger', 'article', 'avoid', 'merged', 'hist...","['merger', 'articl', 'avoid', 'merg', 'histori...",0


## K_FOLD

In [ ]:
%%time
import ast
from sklearn.model_selection import KFold
# K_Fold Cross Validation
df_typo['tokens_raw'] = df_typo['tokens_raw'].apply(lambda x : ast.literal_eval(x))
kf = KFold(n_splits=5,random_state=42)
K_FOLD_LIST = []
k=1
for train_index, test_index in kf.split(df_typo):
  K_FOLD_LIST.append([train_index, test_index,k,df_typo['tokens_raw'].iloc[test_index].apply(len).sum()])
  k+=1
print("train index\t\t\t\t\ttest index\t\t\t\t\tk   word nums")
print(K_FOLD_LIST[0][0],K_FOLD_LIST[0][1],"     ",K_FOLD_LIST[0][2],K_FOLD_LIST[0][3])
print(K_FOLD_LIST[1][0],K_FOLD_LIST[1][1],K_FOLD_LIST[1][2],K_FOLD_LIST[1][3])
print(K_FOLD_LIST[2][0],K_FOLD_LIST[2][1],K_FOLD_LIST[2][2],K_FOLD_LIST[2][3])
print(K_FOLD_LIST[3][0],K_FOLD_LIST[3][1],K_FOLD_LIST[3][2],K_FOLD_LIST[3][3])
print(K_FOLD_LIST[4][0],K_FOLD_LIST[4][1],K_FOLD_LIST[4][2],K_FOLD_LIST[4][3])
print("sequence_length : ",max(df_typo['tokens_raw'].apply(len)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


train index					test index					k   word nums
[ 63561  63562  63563 ... 317799 317800 317801] [    0     1     2 ... 63558 63559 63560]       1 1943148
[     0      1      2 ... 317799 317800 317801] [ 63561  63562  63563 ... 127119 127120 127121] 2 1930106
[     0      1      2 ... 317799 317800 317801] [127122 127123 127124 ... 190679 190680 190681] 3 1944610
[     0      1      2 ... 317799 317800 317801] [190682 190683 190684 ... 254239 254240 254241] 4 1935237
[     0      1      2 ... 254239 254240 254241] [254242 254243 254244 ... 317799 317800 317801] 5 1929481
sequence_length :  235
CPU times: user 19.8 s, sys: 807 ms, total: 20.6 s
Wall time: 20.8 s


## Dataset_list

In [ ]:
dataset_list = [X_cleaned,X_typo]
dataset_name = ["Clean","Typo"]

dataset = {na:li for na,li in zip(dataset_name,dataset_list)}

# Import Models

In [ ]:
pip install transformers

     |████████████████████████████████| 2.5MB 6.5MB/s 
     |████████████████████████████████| 3.3MB 26.8MB/s 
     |████████████████████████████████| 901kB 39.0MB/s 


In [ ]:
Model_list = []

## ALBERT

12 repeating layers

128 embedding dimension

768 hidden dimension

12 attention heads

11M parameters

In [ ]:
from transformers import AlbertTokenizerFast, TFAlbertForSequenceClassification
tokenizer_albert = AlbertTokenizerFast.from_pretrained('albert-base-v2')
model_albert = TFAlbertForSequenceClassification.from_pretrained('albert-base-v2')
Model_list.append((AlbertTokenizerFast,TFAlbertForSequenceClassification,'ALBERT','albert-base-v2','TF'))

All model checkpoint layers were used when initializing TFAlbertForSequenceClassification.

Some layers of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## RoBERTa

In [ ]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
tokenizer_roberta = RobertaTokenizer.from_pretrained('roberta-base')
model_roberta = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
Model_list.append((RobertaTokenizer,TFRobertaForSequenceClassification,'RoBERTa','roberta-base','TF'))

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## DistilBERT

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
tokenizer_distilbert = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_distilbert = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
Model_list.append((DistilBertTokenizer,TFDistilBertForSequenceClassification,'DistilBERT','distilbert-base-uncased','TF'))

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_57', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

## ELECTRA

In [ ]:
from transformers import ElectraTokenizer, TFElectraForSequenceClassification
tokenizer_electra = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
model_electra = TFElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator')
Model_list.append((ElectraTokenizer,TFElectraForSequenceClassification,'ELECTRA','google/electra-small-discriminator','TF'))

Some layers from the model checkpoint at google/electra-small-discriminator were not used when initializing TFElectraForSequenceClassification: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## MobileBERT

In [ ]:
from transformers import MobileBertTokenizer, TFMobileBertForSequenceClassification
tokenizer_mobilebert = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model_mobilebert = TFMobileBertForSequenceClassification.from_pretrained('google/mobilebert-uncased')
Model_list.append((MobileBertTokenizer,TFMobileBertForSequenceClassification,'MobileBERT','google/mobilebert-uncased','TF'))

All model checkpoint layers were used when initializing TFMobileBertForSequenceClassification.

Some layers of TFMobileBertForSequenceClassification were not initialized from the model checkpoint at google/mobilebert-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## SqueezeBERT (PyTorch)

In [ ]:
# from transformers import SqueezeBertTokenizer, SqueezeBertForSequenceClassification
# tokenizer_squeezebert = SqueezeBertTokenizer.from_pretrained('squeezebert/squeezebert-uncased')
# model_squeezebert = SqueezeBertForSequenceClassification.from_pretrained('squeezebert/squeezebert-uncased')
# Model_list.append((tokenizer_squeezebert,model_squeezebert,'SqueezeBERT'))

# TAINING & EVALUATION

In [ ]:
for i in range(len(Model_list)):
  print(f"{i+1} - Tokenizer[0] : {str(Model_list[i][0]).split('.')[-1][:-2]}",end='\t\t')
  print(f"Model[1] : {str(Model_list[i][1]).split('.')[-1][:-2]:38s}",end='\t')
  print(f"Name[2] : {Model_list[i][2]:10s}",end='\t')
  print(f"Pretrained[3] : {Model_list[i][3]}")

1 - Tokenizer[0] : RobertaTokenizer		Model[1] : TFRobertaForSequenceClassification    	Name[2] : RoBERTa   	Pretrained[3] : roberta-base
2 - Tokenizer[0] : DistilBertTokenizer		Model[1] : TFDistilBertForSequenceClassification 	Name[2] : DistilBERT	Pretrained[3] : distilbert-base-uncased
3 - Tokenizer[0] : ElectraTokenizer		Model[1] : TFElectraForSequenceClassification    	Name[2] : ELECTRA   	Pretrained[3] : google/electra-small-discriminator
4 - Tokenizer[0] : MobileBertTokenizer		Model[1] : TFMobileBertForSequenceClassification 	Name[2] : MobileBERT	Pretrained[3] : google/mobilebert-uncased


In [ ]:
import time
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

In [ ]:
eval = pd.DataFrame([[np.nan for i in range(11)]])
eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
eval = eval.iloc[1:]
 
 
for data_name in dataset_name:
  for tokenizer, model_seq, model_name, pretrained, plat in Model_list:
    print(f"Tokenizer[0] : {str(tokenizer).split('.')[-1][:-2]}",end='\t\t')
    print(f"Model[1] : {str(model_seq).split('.')[-1][:-2]:38s}",end='\t')
    print(f"Name[2] : {model_name:10s}",end='\t')
    print(f"Pretrained[3] : {pretrained}")
 
    # Performance Base DataFrame
    LR_list_base = [model_name] # ALBERT
    LR_list_base.append(data_name) # CLEAN
 
    # Load Data
    X = dataset[data_name]
 
    # K_Fold Cross Validation
    for train_index, test_index, k, word_nums in K_FOLD_LIST:
      tokenizer = tokenizer.from_pretrained(pretrained)
      LR = model_seq.from_pretrained(pretrained)
      optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
      LR.compile(optimizer=optimizer, loss=LR.compute_loss) # can also use any keras loss fn
 
 
      # Get Train, Test
      X_train, X_test = X.iloc[train_index].to_list(), X.iloc[test_index].to_list()
      y_train, y_test = y.iloc[train_index].to_list(), y.iloc[test_index].to_list()
 
      encoding_time = time.time()
      train_encodings = tokenizer(X_train, truncation=True, padding=True)
      test_encodings = tokenizer(X_test, truncation=True, padding=True)
      print("encoding time : ", time.time()-encoding_time)
 
      train_dataset = 0
      test_dataset = 0
 
      dataset_time = time.time()
      train_dataset = tf.data.Dataset.from_tensor_slices((
          dict(train_encodings),
          y_train
      ))
      test_dataset = tf.data.Dataset.from_tensor_slices((
          dict(test_encodings),
          y_test
      ))
      print("dataset time : ", time.time()-dataset_time)
 
      # Fit and Inference
      LR.fit(train_dataset.shuffle(1000).batch(16), epochs=5)
 
      print(f"{data_name} - {k} Trained")
      fitted = LR.predict(train_dataset.shuffle(1000).batch(16))
      fitted = tf.nn.softmax(fitted.logits, axis=1)
      fitted_proba = fitted[:,1]
      fitted = [np.argmax(res) for res in fitted]


      start = time.time()
      pred = LR.predict(test_dataset.shuffle(1000).batch(16))
      pred = tf.nn.softmax(pred.logits, axis=1)
      inference_time = time.time()-start
      pred_proba = pred[:,1]
      pred = [np.argmax(res) for res in pred_proba]

      
      print(f"{data_name} - {k} Inferenced : {inference_time}s",end='\t')
 
      # Evaluate
      train_acc = accuracy_score(y_train,fitted)
      train_auc = roc_auc_score(y_train,fitted_proba)
      train_f1 = f1_score(y_train,fitted)
      test_acc = accuracy_score(y_test,pred)
      test_auc = roc_auc_score(y_test,pred_proba)
      test_f1 = f1_score(y_test,pred)
      print(f"train ACC : {train_acc} test ACC : {test_acc} test F1 : {test_f1}")
 
      LR_list = LR_list_base.copy()
      LR_list.append(k)
      LR_list.append(train_acc)
      LR_list.append(train_auc)
      LR_list.append(train_f1)
      LR_list.append(test_acc)
      LR_list.append(test_auc)
      LR_list.append(test_f1)
      LR_list.append(inference_time)
      LR_list.append(word_nums)
 
      eval = eval.append(pd.DataFrame([LR_list],columns=eval.columns))
      eval.to_csv(f"/content/drive/MyDrive/HateSpeech/PERFORMANCE/{model_name}_performance_{k}.csv")
      eval = pd.DataFrame([[np.nan for i in range(11)]])
      eval.columns = ['Model','Dataset','K_fold','Train_Score(ACC)','Train_Score(ROC_AUC)','Train_Score(F1)','Test_Score(ACC)','Test_Score(ROC_AUC)','Test_Score(F1)','Inference_Time','words_nums']
      eval = eval.iloc[1:]

# SIMPLE TEST

In [ ]:
inputs = "Fuck you Ass hole you are bueatiful!"
inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors="tf")
tf.nn.softmax(model(inputs)[0].numpy(), axis=1).numpy()[0]

array([0.13482197, 0.86517805], dtype=float32)

In [ ]:
inputs = "you are bueatiful!"
inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors="tf")
tf.nn.softmax(model(inputs)[0].numpy(), axis=1).numpy()[0]

array([0.74221206, 0.25778797], dtype=float32)

In [ ]:
inputs = "I thi like  fe fewp nk it's so fucking bueatiful voice. I cunta love this is love music so much. It's my favorite."
inputs = tokenizer(inputs, truncation=True, padding=True, return_tensors="tf")
tf.nn.softmax(model(inputs)[0].numpy(), axis=1).numpy()[0]

array([0.44029763, 0.55970234], dtype=float32)